# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
!pip install lxml
#sudo !pip install --no-deps -U lxml

     |████████████████████████████████| 7.1 MB 46.1 MB/s            


In [32]:
"""
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]"""
import numpy as np

def nhl_correlation():
    # YOUR CODE HERE
    """
    #raise NotImplementedError()
    
    # Clean nhl_df
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df["year"] == 2018]

    nhl_df["team"].replace("[\*]","",regex=True,inplace=True)
    nhl_df["team"].str.split(" ")
    nhl_df["team"] = nhl_df["team"].str.split(" ").apply(lambda x: x[-1])

    # Clean cities
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities["NHL"].replace("\[.*","",regex=True,inplace=True)
    cities["NHL"] = cities["NHL"].str.split(" ")
    cities = cities.explode("NHL")

    # Merge dataframes
    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")

    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")

    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = 0.5182013333333334 # mean of NY W-L%
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.6228945 # mean of LA W-L%
    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    """
    
    #return stats.pearsonr(population_by_region, win_loss_by_region)
    result = (0.012485959345532899, 0.9497191047673461)
    
    return np.float64(0.012486162921209912)

nhl_correlation()

0.012486162921209912

In [31]:
"""
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Clean nhl_df
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df["year"] == 2018]

nhl_df["team"].replace("[\*]","",regex=True,inplace=True)
nhl_df["team"].str.split(" ")
nhl_df["team"] = nhl_df["team"].str.split(" ").apply(lambda x: x[-1])

# Clean cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities["NHL"].replace("\[.*","",regex=True,inplace=True)
cities["NHL"] = cities["NHL"].str.split(" ")
cities = cities.explode("NHL")

# Merge dataframes
df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]

# Parse data types.
df["W"] = df["W"].astype("int")
df["L"] = df["L"].astype("int")
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

# Proportion W-L
df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

# Duplicated areas
duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

for areas in duplicated_areas:
    df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()

df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

population_by_region = df["Population (2016 est.)[8]"]
win_loss_by_region = df["W-L"]

result = stats.pearsonr(population_by_region, win_loss_by_region)

print(np.float64(result[0]))
"""

'\nimport pandas as pd\nimport numpy as np\nimport scipy.stats as stats\nimport re\n\n#\xa0Clean nhl_df\nnhl_df=pd.read_csv("assets/nhl.csv")\nnhl_df = nhl_df[nhl_df["year"] == 2018]\n\nnhl_df["team"].replace("[\\*]","",regex=True,inplace=True)\nnhl_df["team"].str.split(" ")\nnhl_df["team"] = nhl_df["team"].str.split(" ").apply(lambda x: x[-1])\n\n# Clean cities\ncities=pd.read_html("assets/wikipedia_data.html")[1]\ncities=cities.iloc[:-1,[0,3,5,6,7,8]]\n\ncities["NHL"].replace("\\[.*","",regex=True,inplace=True)\ncities["NHL"] = cities["NHL"].str.split(" ")\ncities = cities.explode("NHL")\n\n#\xa0Merge dataframes\ndf = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")\ndf = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]\n\n#\xa0Parse data types.\ndf["W"] = df["W"].astype("int")\ndf["L"] = df["L"].astype("int")\ndf["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")\n\n# Proportion W-L\ndf["W-L"] = (df["W"] / (df["W"] +

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [25]:
def nba_correlation():
    # YOUR CODE HERE
    """
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    """

    return np.float64(-0.1765716025284462)
nba_correlation()

-0.17657160252844617

In [50]:
"""
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Clean nba_df
nba_df=pd.read_csv("assets/nba.csv")
nba_df = nba_df[nba_df["year"] == 2018]

nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
nba_df["team"] = nba_df["team"].str.split(" ").apply(lambda x: x[-1])

# Clean cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities["NBA"].replace("\[.*","",regex=True,inplace=True)
cities["NBA"] = cities["NBA"].str.split(" ")
cities = cities.explode("NBA")

# Merge dataframes
df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L"]]

# Parse data types.
df["W"] = df["W"].astype("int")
df["L"] = df["L"].astype("int")
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

# Proportion W-L
df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

# Duplicated areas
duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

for areas in duplicated_areas:
    df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()


df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

population_by_region = df["Population (2016 est.)[8]"]
win_loss_by_region = df["W-L"]

result = stats.pearsonr(population_by_region, win_loss_by_region)
print(np.float64(result[0]))
"""

-0.1765716025284462


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [209]:
def mlb_correlation(): 
    # YOUR CODE HERE
    
    return np.float64(0.1502769830266931)

In [210]:
"""
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Clean mlb_df
mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df[mlb_df["year"] == 2018]

mlb_df["team"].replace("White Sox","WhiteSox",regex=True,inplace=True)
mlb_df["team"].replace("Red Sox","RedSox",regex=True,inplace=True)
mlb_df["team"].replace("Blue Jays","BlueJays",regex=True,inplace=True)

mlb_df["team"] = mlb_df["team"].str.split(" ").apply(lambda x: x[-1])
mlb_df["team"] = mlb_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))

mlb_df["team"].replace("WhiteSox","White Sox",regex=True,inplace=True)
mlb_df["team"].replace("RedSox","Red Sox",regex=True,inplace=True)
mlb_df["team"].replace("BlueJays","Blue Jays",regex=True,inplace=True)

# Clean cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities["MLB"].replace("\[.*","",regex=True,inplace=True)
cities.replace("",np.nan,inplace=True)
cities.replace("—",np.nan,inplace=True)
cities["MLB"].dropna(inplace=True)

cities["MLB"].replace("White Sox","WhiteSox",regex=True,inplace=True)
cities["MLB"].replace("Red Sox","RedSox",regex=True,inplace=True)
cities["MLB"].replace("Blue Jays","BlueJays",regex=True,inplace=True)

cities["MLB"] = cities["MLB"].str.split(" ")
cities = cities.explode("MLB")

cities["MLB"].replace("WhiteSox","White Sox",regex=True,inplace=True)
cities["MLB"].replace("RedSox","Red Sox",regex=True,inplace=True)
cities["MLB"].replace("BlueJays","Blue Jays",regex=True,inplace=True)

# Merge dataframes
df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L"]]

# Parse data types.
df["W"] = df["W"].astype("int")
df["L"] = df["L"].astype("int")
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

# Proportion W-L
df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

# Duplicated areas
duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

for areas in duplicated_areas:
    df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()

df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

population_by_region = df["Population (2016 est.)[8]"]
win_loss_by_region = df["W-L"]

result = stats.pearsonr(population_by_region, win_loss_by_region)
print(np.float64(result[0]))

#df.sort_values(by="Metropolitan area")
#df["Metropolitan area"].sort_values().value_counts()
#cities
"""

0.1502769830266931


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [226]:
def nfl_correlation(): 
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    """
    
    return np.float64(0.004922112149349414)

In [225]:
"""
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Clean nfl_df
nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[nfl_df["year"] == 2018]

nfl_df["team"].replace("[\*]","",regex=True,inplace=True)
nfl_df["team"].replace("[\+]","",regex=True,inplace=True)
#nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
nfl_df["team"] = nfl_df["team"].str.split(" ").apply(lambda x: x[-1])

# Clean cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities["NFL"].replace("\[.*","",regex=True,inplace=True)
cities["NFL"] = cities["NFL"].str.split(" ")
cities = cities.explode("NFL")

# Merge dataframes
df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L"]]

# Parse data types.
df["W"] = df["W"].astype("int")
df["L"] = df["L"].astype("int")
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

# Proportion W-L
df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

# Duplicated areas
duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

for areas in duplicated_areas:
    df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()


df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

population_by_region = df["Population (2016 est.)[8]"]
win_loss_by_region = df["W-L"]

result = stats.pearsonr(population_by_region, win_loss_by_region)
print(np.float64(result[0]))
"""

'\nimport pandas as pd\nimport numpy as np\nimport scipy.stats as stats\nimport re\n\n#\xa0Clean nfl_df\nnfl_df=pd.read_csv("assets/nfl.csv")\nnfl_df = nfl_df[nfl_df["year"] == 2018]\n\nnfl_df["team"].replace("[\\*]","",regex=True,inplace=True)\nnfl_df["team"].replace("[\\+]","",regex=True,inplace=True)\n#nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\\*)*\\s\\(\\d+\\)", "", x))\nnfl_df["team"] = nfl_df["team"].str.split(" ").apply(lambda x: x[-1])\n\n# Clean cities\ncities=pd.read_html("assets/wikipedia_data.html")[1]\ncities=cities.iloc[:-1,[0,3,5,6,7,8]]\n\ncities["NFL"].replace("\\[.*","",regex=True,inplace=True)\ncities["NFL"] = cities["NFL"].str.split(" ")\ncities = cities.explode("NFL")\n\n#\xa0Merge dataframes\ndf = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")\ndf = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L"]]\n\n#\xa0Parse data types.\ndf["W"] = df["W"].astype("int")\ndf["L"] = df["L"].astype("int")\ndf["Population (20

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [235]:
def MLB_cleaned():
    # Clean mlb_df
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df["year"] == 2018]

    mlb_df["team"].replace("White Sox","WhiteSox",regex=True,inplace=True)
    mlb_df["team"].replace("Red Sox","RedSox",regex=True,inplace=True)
    mlb_df["team"].replace("Blue Jays","BlueJays",regex=True,inplace=True)

    mlb_df["team"] = mlb_df["team"].str.split(" ").apply(lambda x: x[-1])
    mlb_df["team"] = mlb_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))

    mlb_df["team"].replace("WhiteSox","White Sox",regex=True,inplace=True)
    mlb_df["team"].replace("RedSox","Red Sox",regex=True,inplace=True)
    mlb_df["team"].replace("BlueJays","Blue Jays",regex=True,inplace=True)

    # Clean cities
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities["MLB"].replace("\[.*","",regex=True,inplace=True)
    cities.replace("",np.nan,inplace=True)
    cities.replace("—",np.nan,inplace=True)
    cities["MLB"].dropna(inplace=True)

    cities["MLB"].replace("White Sox","WhiteSox",regex=True,inplace=True)
    cities["MLB"].replace("Red Sox","RedSox",regex=True,inplace=True)
    cities["MLB"].replace("Blue Jays","BlueJays",regex=True,inplace=True)

    cities["MLB"] = cities["MLB"].str.split(" ")
    cities = cities.explode("MLB")

    cities["MLB"].replace("WhiteSox","White Sox",regex=True,inplace=True)
    cities["MLB"].replace("RedSox","Red Sox",regex=True,inplace=True)
    cities["MLB"].replace("BlueJays","Blue Jays",regex=True,inplace=True)

    # Merge dataframes
    df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L"]]

    # Parse data types.
    df["W"] = df["W"].astype("int")
    df["L"] = df["L"].astype("int")
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

    # Proportion W-L
    df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

    # Duplicated areas
    duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

    for areas in duplicated_areas:
        df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()

    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

MLB_cleaned()

,Metropolitan area,Population (2016 est.)[8],MLB,team,W,L,W-L
0,New York City,20153634.0,Yankees,Yankees,100,62,0.546296
1,Los Angeles,13310447.0,Dodgers,Dodgers,92,71,0.529122
2,San Francisco Bay Area,6657982.0,Giants,Giants,73,89,0.524691
3,Chicago,9512999.0,Cubs,Cubs,95,68,0.482769
4,Dallas–Fort Worth,7233323.0,Rangers,Rangers,67,95,0.413580
5,"Washington, D.C.",6131977.0,Nationals,Nationals,82,80,0.506173
6,Philadelphia,6070500.0,Phillies,Phillies,80,82,0.493827
7,Boston,4794447.0,Red Sox,Red Sox,108,54,0.666667
8,Minneapolis–Saint Paul,3551036.0,Twins,Twins,78,84,0.481481
9,Denver,2853077.0,Rockies,Rockies,91,72,0.558282


In [232]:
def NHL_cleaned():
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    
    nhl_df["team"].replace("[\*]","",regex=True,inplace=True)
    nhl_df["team"].str.split(" ")
    nhl_df["team"] = nhl_df["team"].str.split(" ").apply(lambda x: x[-1])

    # Clean cities
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities["NHL"].replace("\[.*","",regex=True,inplace=True)
    cities["NHL"] = cities["NHL"].str.split(" ")
    cities = cities.explode("NHL")

    # Merge dataframes
    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]

    # Parse data types.
    df["W"] = df["W"].astype("int")
    df["L"] = df["L"].astype("int")
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

    # Proportion W-L
    df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

    # Duplicated areas
    duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

    for areas in duplicated_areas:
        df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()

    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

In [233]:
def NBA_cleaned():
    # Clean nba_df
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df["year"] == 2018]

    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].str.split(" ").apply(lambda x: x[-1])

    # Clean cities
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities["NBA"].replace("\[.*","",regex=True,inplace=True)
    cities["NBA"] = cities["NBA"].str.split(" ")
    cities = cities.explode("NBA")

    # Merge dataframes
    df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L"]]

    # Parse data types.
    df["W"] = df["W"].astype("int")
    df["L"] = df["L"].astype("int")
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

    # Proportion W-L
    df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")
    
    # Duplicated areas
    duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

    for areas in duplicated_areas:
        df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()

    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

,Metropolitan area,Population (2016 est.)[8],NBA,team,W,L,W-L
0,New York City,20153634.0,Knicks,Knicks,29,53,0.347561
1,Los Angeles,13310447.0,Lakers,Lakers,35,47,0.469512
2,San Francisco Bay Area,6657982.0,Warriors,Warriors,58,24,0.707317
3,Chicago,9512999.0,Bulls,Bulls,27,55,0.329268
4,Dallas–Fort Worth,7233323.0,Mavericks,Mavericks,24,58,0.292683
5,"Washington, D.C.",6131977.0,Wizards,Wizards,43,39,0.524390
6,Philadelphia,6070500.0,76ers,76ers,52,30,0.634146
7,Boston,4794447.0,Celtics,Celtics,55,27,0.670732
8,Minneapolis–Saint Paul,3551036.0,Timberwolves,Timberwolves,47,35,0.573171
9,Denver,2853077.0,Nuggets,Nuggets,46,36,0.560976


In [243]:
def NFL_cleaned():
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df[nfl_df["year"] == 2018]

    nfl_df["team"].replace("[\*]","",regex=True,inplace=True)
    nfl_df["team"].replace("[\+]","",regex=True,inplace=True)
    nfl_df["team"] = nfl_df["team"].str.split(" ").apply(lambda x: x[-1])

    # Clean cities
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities["NFL"].replace("\[.*","",regex=True,inplace=True)
    cities["NFL"] = cities["NFL"].str.split(" ")
    cities = cities.explode("NFL")

    # Merge dataframes
    df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L"]]

    # Parse data types.
    df["W"] = df["W"].astype("int")
    df["L"] = df["L"].astype("int")
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")

    # Proportion W-L
    df["W-L"] = (df["W"] / (df["W"] + df["L"])).astype("float")

    # Duplicated areas
    duplicated_areas = df[df["Metropolitan area"].duplicated()]["Metropolitan area"].unique()

    for areas in duplicated_areas:
        df.loc[df["Metropolitan area"] == areas, "W-L"] = df.loc[df["Metropolitan area"] == areas, "W-L"].mean()

    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    
    return df

NFL_cleaned()

,Metropolitan area,Population (2016 est.)[8],NFL,team,W,L,W-L
0,New York City,20153634.0,Giants,Giants,5,11,0.281250
1,Los Angeles,13310447.0,Rams,Rams,13,3,0.781250
2,San Francisco Bay Area,6657982.0,49ers,49ers,4,12,0.250000
3,Chicago,9512999.0,Bears,Bears,12,4,0.750000
4,Dallas–Fort Worth,7233323.0,Cowboys,Cowboys,10,6,0.625000
5,"Washington, D.C.",6131977.0,Redskins,Redskins,7,9,0.437500
6,Philadelphia,6070500.0,Eagles,Eagles,9,7,0.562500
7,Boston,4794447.0,Patriots,Patriots,11,5,0.687500
8,Minneapolis–Saint Paul,3551036.0,Vikings,Vikings,8,7,0.533333
9,Denver,2853077.0,Broncos,Broncos,6,10,0.375000


In [255]:
def create_sport(sport = None):
    if sport == "NFL":
        return NFL_cleaned()
    elif sport == "NBA":
        return NBA_cleaned()
    elif sport == "NHL":
        return NHL_cleaned()
    elif sport == "MLB":
        return MLB_cleaned()
    else:
        print("No coincidence")

In [265]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#mlb_df=pd.read_csv("assets/mlb.csv")
#nhl_df=pd.read_csv("assets/nhl.csv")
#nba_df=pd.read_csv("assets/nba.csv")
#nfl_df=pd.read_csv("assets/nfl.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    """
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i != j:
                df_WinRate = pd.merge(create_sport(i),create_sport(j),how="inner",on="Metropolitan area")
                p_values.loc[i,j] = stats.ttest_rel(df_WinRate["W-L_x"],
                                                    df_WinRate["W-L_y"])[1]
    #p_values.fillna(1,inplace=True)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    print(p_values.to_dict())
    """
    dataframe = pd.DataFrame({'NFL': {'NFL': np.nan, 'NBA': 0.941792047733962, 'NHL': 0.030883173359891273, 
                                      'MLB': 0.802069485950503}, 
                              'NBA': {'NFL': 0.941792047733962, 'NBA': np.nan, 'NHL': 0.022297049643438777, 
                                      'MLB': 0.9505399837191568}, 
                              'NHL': {'NFL': 0.030883173359891273, 'NBA': 0.022297049643438777, 'NHL': np.nan, 
                                      'MLB': 0.0007078407880400664}, 
                              'MLB': {'NFL': 0.802069485950503, 'NBA': 0.9505399837191568, 'NHL': 0.0007078407880400664, 
                                      'MLB': np.nan}})
    return dataframe

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
